# LlamaIndex 기반 JSON 준비 파이프라인 설계 설명서

### 1. 목적과 범위

이 노트북의 목적은 MongoDB에 저장된 뉴스 문서를 LlamaIndex가 다루기 쉬운 형식으로 정리하여,    
추후 검색·요약·질의응답 등 다운스트림 작업에 바로 활용할 수 있도록 JSON 파일로 준비하는 것이다.    

* 데이터 원본: MongoDB polaris 데이터베이스, yna_preprocessed_v2 컬렉션
* 처리 대상 기간: 코드에서 시작일/종료일을 지정해 재현 가능한 기간 단위 수집을 수행
* 산출물: LlamaIndex의 Document 혹은 Node(청크)를 UTF-8 JSON으로 저장    
    
이렇게 만들어둔 JSON은 이후 LLM VM 환경으로 옮겨 곧바로 인덱싱·임베딩·검색에 사용할 수 있다.

### 2. 입력 데이터 로드와 기본 전처리

#### 2.1 MongoDB 연결 및 기간 필터

* 연결 정보와 컬렉션명을 코드 상단에서 명시한다.
* 시작일·종료일을 datetime으로 지정하여 기간 내 문서만 로드한다.
* 이유
    * 기간을 명시하면 실험 재현성이 높아진다.
    * LLM 기반 처리는 비용이 크므로 필요 범위만 다루는 것이 효율적이다.
    * 시계열 분석 시 기간별 성능 비교가 쉬워진다.

#### 2.2 예외 처리와 자원 정리

* 연결 실패, 쿼리 실패 등에 대비해 예외 처리를 적용했다.
* 작업 종료 후 client.close()로 연결을 명시적으로 종료한다.
* 이유
    * 장기 실행 시 연결 누수 방지
    * 운영 환경에서의 안정성 확보

## 3. LlamaIndex 문서 객체로의 변환

#### 3.1 Document 객체로 통합
* MongoDB 문서에서 content(본문)와 title(제목)을 꺼내 **LlamaIndex Document**로 변환한다.
* metadata에는 제목, 발행일(pubDate), URL, 카테고리를 넣는다.
* 문자열이 아닐 수 있는 필드는 안전하게 문자열로 변환하여 타입 문제를 예방한다.

#### 3.2 이렇게 한 이유
* LlamaIndex의 후속 단계(청킹, 임베딩, 검색)는 Document/Node 중심으로 동작한다.
* 메타데이터를 보존하면 검색 결과에서 출처·제목·URL을 바로 보여줄 수 있고, 이후 카테고리·날짜 필터링에 사용할 수 있다.
* 타입을 엄격히 맞추면 예상치 못한 파싱 오류를 줄일 수 있다.

## 4. 문서 청킹(Node) 설계(선택 사용)

#### 4.1 청킹 기본 구상
* SentenceSplitter로 **청크 크기(예: 512)와 겹침(예: 20)**을 설정해 문서를 작은 단위로 나눈다.
* 현재 노트북에서는 청킹 부분이 주석 처리되어 있으며, 필요할 때 활성화해서 쓴다.

#### 4.2 이렇게 한 이유
* 검색·재랭킹·질의응답은 짧은 단위 텍스트에서 성능이 잘 나온다.
* 청크가 너무 길면 모델 컨텍스트를 낭비하고, 너무 짧으면 의미가 끊긴다.
* 겹침을 주면 문장 경계에서 의미 단절을 완화할 수 있다.

## 5. 직렬화와 저장(산출물 만들기)

#### 5.1 저장 형식과 파일
* 문서 단위 저장: prepared_nodes_for_llm_vm.json
* 청크 단위 저장(선택): prepared_nodes_for_llm_vm_6_30.json
* 저장은 ensure_ascii=False, indent=2로 진행하여 한글을 보존하고 사람이 읽기 쉽다.

#### 5.2 불필요 필드 정리
* Node.to_dict() 결과에서 임베딩이나 미사용 리소스(이미지·오디오·비디오 등)가 None이거나 불필요할 경우 제거하도록 정리 함수가 포함되어 있다.
* 이렇게 하면 파일 크기를 줄이고, JSON 직렬화 호환성을 높이며, 이후 환경에서 임베딩 모델을 자유롭게 교체하기 쉽다(임베딩은 나중에 생성).

## 6. 설계 선택의 핵심 근거

#### 1. 기간 필터
* 비용과 시간 관리 차원에서 필수적이다.
* 분석·평가를 기간 단위로 반복하기 쉽다.

#### 2. Document 중심 표준화
* LlamaIndex의 표준 입력 형태를 맞추면 이후 단계가 단순해진다.
* 메타데이터가 보존되어 출처 추적과 필터링이 용이하다.

#### 3. 청킹의 선택적 적용
* 초기에는 문서 단위 저장으로 빠르게 준비하고, 필요 시 청킹을 켜서 검색·Q&A 품질을 올린다.

#### 4. 임베딩 후생성 전략
* 저장 단계에서는 임베딩을 포함하지 않는다.
* 환경에 따라 임베딩 모델을 바꾸거나 재생성할 수 있어 유연성이 높다.

#### 5. 안전한 직렬화
* 한글 보존, 가독성, 불필요 필드 제거로 운영에 부담이 적다.

## 7. 실행 순서 요약

1. 기간 설정: 시작일·종료일을 지정한다.

2. 데이터 로드: MongoDB에서 해당 기간 문서를 불러온다.

3. 문서 변환: 본문과 메타데이터로 Document를 생성한다.

4. (선택) 청킹: 필요 시 SentenceSplitter를 활성화해 Node를 만든다.

5. 저장: 문서 단위 또는 청크 단위를 UTF-8 JSON으로 저장한다.